In [1]:
import torch
import torch.nn
import torchvision
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torchvision import models
import cv2
import os
import pandas as pd
import numpy as np
import gc
from PIL import Image
from sklearn.model_selection import train_test_split
metadata_dir=r"/Users/severex_u_r1/Ipython/pytorch_train/metadata/"
photo_dir=r"/Users/severex_u_r1/Ipython/pytorch_train/HAM10000_images/"
all_images_paths=os.listdir(photo_dir)


In [2]:
%%time
from torch.utils.data import Dataset,DataLoader
class CustomDataSet(Dataset):
    def __init__(self, main_dir, transform=None):
        self.main_dir = main_dir
        all_imgs = os.listdir(main_dir)
        self.total_imgs = all_imgs
        self.transform=transform

    def __len__(self):
        return len(self.total_imgs)

    def __getitem__(self, idx):
        img_loc = os.path.join(self.main_dir, self.total_imgs[idx])
        image = cv2.imread(img_loc)
        if self.transform:
            image=self.transform(image)
        return image
    
    
def compute_mean(input_size):
    im_dataset=CustomDataSet(photo_dir,transforms.Compose([transforms.ToPILImage(),
                                                           transforms.Resize((input_size,input_size)),
                                                          transforms.ToTensor()]))
    im_loader=DataLoader(im_dataset,batch_size=16)
    all_mean=[]
    all_std=[]
    for img in im_loader:
        batch_img=img.numpy()
#        batch_img=batch_img.astype(np.float32) / 255.
        all_mean.append(np.mean(batch_img,axis=(0,2,3)))
        all_std.append(np.std(batch_img,axis=(0,2,3)))
    return np.mean(all_mean,axis=0),np.mean(all_std,axis=0)
metadata=pd.read_csv(os.path.join(metadata_dir,"metadata.csv"))
metadata['path']=metadata['image_id'].apply(lambda x: os.path.join(photo_dir,x+".jpg"))
metadata['label']=pd.Categorical(metadata['dx']).codes
num_classes=len(metadata['dx'].unique())
#all_mean,all_std=compute_mean(input_size=224)

CPU times: user 41.6 ms, sys: 6.55 ms, total: 48.2 ms
Wall time: 47.7 ms


In [3]:
all_mean=[0.485, 0.456, 0.406]
all_std=[0.229, 0.224, 0.225]
df_train,df_val=train_test_split(metadata,test_size=0.2,random_state=100,stratify=metadata['dx'])
df_train=df_train.reset_index(drop=True)
df_val=df_val.reset_index(drop=True)

In [13]:
from torch import nn
from torch import optim
def create_model():
    model=models.densenet121(pretrained=True)
    for param in model.parameters():
            param.requires_grad = False
    num_last_neur=model.classifier.in_features
    model.classifier=nn.Linear(num_last_neur,num_classes)
    return model
model=create_model()
for param in model.features.denseblock4.parameters():
    param.requires_grad=True
for param in model.features.denseblock3.parameters():
    param.requires_grad=True
device=torch.device("cpu:0")
model=model.to(device)
input_size=224
optimizer = optim.Adam(model.parameters())
criterion = nn.CrossEntropyLoss().to(device)

In [14]:
class main_dataset(Dataset):
    def __init__(self,df,transform=None):
        self.df=df
        self.transform=transform
    def __len__(self):
        return len(self.df)
    def __getitem__(self,index):
        img=Image.open(self.df['path'][index])
        label=torch.tensor(self.df['label'][index], dtype=torch.long)
        if self.transform:
            img=self.transform(img)
        return img,label
    
    
train_transform = transforms.Compose([transforms.Resize((input_size,input_size)),
                                      transforms.RandomVerticalFlip(),
                                      transforms.RandomRotation(30),
                                      transforms.ColorJitter(brightness=0.1, contrast=0.05, hue=0.05),
                                      transforms.RandomHorizontalFlip(),
                                      transforms.ToTensor(),
                                      transforms.Normalize(all_mean, all_std)])
val_transform = transforms.Compose([transforms.Resize((input_size,input_size)),
                                    transforms.ToTensor(),
                                    transforms.Normalize(all_mean, all_std)])
train_dataset=main_dataset(df_train,transform=train_transform)
train_loader=DataLoader(train_dataset,batch_size=16,shuffle=True,num_workers=4)
val_dataset=main_dataset(df_val,transform=val_transform)
val_loader=DataLoader(val_dataset,batch_size=16,num_workers=4)

In [15]:
def train(train_loader,model,criterion,optimizer,epoch):
    model.train()
    train_loss=[]
    train_acc=[]
    for itr,data in enumerate(train_loader):
        images,label=data
        batch_size=images.size(0)
        optimizer.zero_grad()
        output=model(images)
        loss=criterion(output,label)
        loss.backward()
        optimizer.step()
        prediction=output.max(1, keepdim=True)[1]
        train_loss.append(loss.item())
        train_acc.append(prediction.eq(label.view_as(prediction)).sum().item()/batch_size)
        if (itr+1)%100==0:
            print("Epoch %d ; Iter %d / %d ; train_loss %.5f ; train_acc %.5f " 
                 %(epoch,itr+1,len(train_loader),sum(train_loss)/len(train_loss),sum(train_acc)/len(train_acc)))
    
    return sum(train_loss)/len(train_loss),sum(train_acc)/len(train_acc)

def val(val_loader,model,criterion,optimizer,epoch):
    model.eval()
    val_loss=[]
    val_acc=[]
    with torch.no_grad():
        for itr,data in enumerate(val_loader):
            images,label=data
            batch_size = images.size(0)
            output = model(images)
            prediction = output.max(1, keepdim=True)[1]
            val_loss.append(criterion(output,label).item())
            val_acc.append(prediction.eq(label.view_as(prediction)).sum().item()/batch_size)
    print("Epoch %d ; val_loss %.5f ; val_acc %.5f " 
             %(epoch,sum(val_loss)/len(val_loss),sum(val_acc)/len(val_acc)))
    return sum(val_loss)/len(val_loss),sum(val_acc)/len(val_acc)
        
            

In [16]:
epochs=110
best_val_acc=0
for epoch in range(1,epochs+1):
    loss_train,acc_train=train(train_loader, model, criterion, optimizer, epoch)
    loss_val,acc_val=val(val_loader, model, criterion, optimizer, epoch)
    if acc_val>best_val_acc:
        best_val_acc=acc_val
        print("BEST RECORD: Epoch %d ; val_loss %.5f ; val_acc %.5f" %(epoch,loss_val,acc_val))
        torch.save({
            "epoch":epoch,
            "model_state_dict":model.state_dict(),
            "optimizer_state_dict":optimizer.state_dict(),
            "loss":loss_train
        },os.path.join(metadata_dir,f"best_model_{round(acc_val,4)}.pt"))

Epoch 1 ; Iter 100 / 501 ; train_loss 0.93047 ; train_acc 0.66625 
Epoch 1 ; Iter 200 / 501 ; train_loss 0.84766 ; train_acc 0.69031 
Epoch 1 ; Iter 300 / 501 ; train_loss 0.80230 ; train_acc 0.71083 
Epoch 1 ; Iter 400 / 501 ; train_loss 0.78111 ; train_acc 0.71828 
Epoch 1 ; Iter 500 / 501 ; train_loss 0.76144 ; train_acc 0.72313 
Epoch 1 ; val_loss 0.64845 ; val_acc 0.78059 
BEST RECORD: Epoch 1 ; val_loss 0.64845 ; val_acc 0.78059
Epoch 2 ; Iter 100 / 501 ; train_loss 0.60186 ; train_acc 0.78375 
Epoch 2 ; Iter 200 / 501 ; train_loss 0.60887 ; train_acc 0.77938 
Epoch 2 ; Iter 300 / 501 ; train_loss 0.60022 ; train_acc 0.78333 
Epoch 2 ; Iter 400 / 501 ; train_loss 0.60089 ; train_acc 0.78375 
Epoch 2 ; Iter 500 / 501 ; train_loss 0.59544 ; train_acc 0.78550 
Epoch 2 ; val_loss 0.55573 ; val_acc 0.80936 
BEST RECORD: Epoch 2 ; val_loss 0.55573 ; val_acc 0.80936
Epoch 3 ; Iter 100 / 501 ; train_loss 0.57632 ; train_acc 0.78500 
Epoch 3 ; Iter 200 / 501 ; train_loss 0.57051 ; train_a

KeyboardInterrupt: 

In [12]:
#continue
PATH="/Users/severex_u_r1/Ipython/pytorch_train/metadata/best_model_0.7289578111946533.pt"
checkpoint = torch.load(PATH)
model.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
epoch = checkpoint['epoch']
epochs=11
best_val_acc = 0.7289578111946533
while epoch <epochs:
    epoch+=1
    loss_train,acc_train=train(train_loader, model, criterion, optimizer, epoch)
    loss_val,acc_val=val(val_loader, model, criterion, optimizer, epoch)
    if acc_val>best_val_acc:
        best_val_acc=acc_val
        print("BEST RECORD: Epoch %d ; val_loss %.5f ; val_acc %.5f" %(epoch,loss_val,acc_val))
        torch.save({
            "epoch":epoch,
            "model_state_dict":model.state_dict(),
            "optimizer_state_dict":optimizer.state_dict(),
            "loss":best_val_acc
        },os.path.join(metadata_dir,f"best_model_{round(acc_val,4)}.pt"))

Epoch 2 ; Iter 100 / 501 ; train_loss 0.77626 ; train_acc 0.71937 
Epoch 2 ; Iter 200 / 501 ; train_loss 0.76524 ; train_acc 0.72781 
Epoch 2 ; Iter 300 / 501 ; train_loss 0.74587 ; train_acc 0.73292 
Epoch 2 ; Iter 400 / 501 ; train_loss 0.74326 ; train_acc 0.73203 
Epoch 2 ; Iter 500 / 501 ; train_loss 0.74315 ; train_acc 0.72975 
Epoch 2 ; val_loss 0.72558 ; val_acc 0.73148 
BEST RECORD: Epoch 2 ; val_loss 0.72558 ; val_acc 0.73148
Epoch 3 ; Iter 100 / 501 ; train_loss 0.67215 ; train_acc 0.75062 
Epoch 3 ; Iter 200 / 501 ; train_loss 0.67335 ; train_acc 0.75219 
Epoch 3 ; Iter 300 / 501 ; train_loss 0.67221 ; train_acc 0.75167 
Epoch 3 ; Iter 400 / 501 ; train_loss 0.68466 ; train_acc 0.74813 
Epoch 3 ; Iter 500 / 501 ; train_loss 0.69105 ; train_acc 0.74638 
Epoch 3 ; val_loss 0.70604 ; val_acc 0.73396 
BEST RECORD: Epoch 3 ; val_loss 0.70604 ; val_acc 0.73396


Traceback (most recent call last):
  File "/Users/severex_u_r1/opt/anaconda3/lib/python3.7/multiprocessing/queues.py", line 242, in _feed
    send_bytes(obj)
  File "/Users/severex_u_r1/opt/anaconda3/lib/python3.7/multiprocessing/connection.py", line 200, in send_bytes
    self._send_bytes(m[offset:offset + size])
Traceback (most recent call last):
  File "/Users/severex_u_r1/opt/anaconda3/lib/python3.7/multiprocessing/connection.py", line 404, in _send_bytes
    self._send(header + buf)
  File "/Users/severex_u_r1/opt/anaconda3/lib/python3.7/multiprocessing/connection.py", line 368, in _send
    n = write(self._handle, buf)
BrokenPipeError: [Errno 32] Broken pipe
  File "/Users/severex_u_r1/opt/anaconda3/lib/python3.7/multiprocessing/queues.py", line 242, in _feed
    send_bytes(obj)
  File "/Users/severex_u_r1/opt/anaconda3/lib/python3.7/multiprocessing/connection.py", line 200, in send_bytes
    self._send_bytes(m[offset:offset + size])
  File "/Users/severex_u_r1/opt/anaconda3/lib/

KeyboardInterrupt: 

In [14]:
all_mean=[0.485, 0.456, 0.406]
all_std=[0.229, 0.224, 0.225]

_DenseBlock(
  (denselayer1): _DenseLayer(
    (norm1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu1): ReLU(inplace=True)
    (conv1): Conv2d(256, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (norm2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu2): ReLU(inplace=True)
    (conv2): Conv2d(128, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  )
  (denselayer2): _DenseLayer(
    (norm1): BatchNorm2d(288, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu1): ReLU(inplace=True)
    (conv1): Conv2d(288, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (norm2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu2): ReLU(inplace=True)
    (conv2): Conv2d(128, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  )
  (denselayer3): _DenseLayer(
    (norm1): BatchNorm2d(320, eps=1e-05, moment